<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Assignment 4 - MapReduce and Spark</center>
<hr style=" border:none; height:3px;">
</h2>

# Introduction


<p align="justify">
<font size="3">
In this exercise you is asked to use Spark for implementing an algorithm that applies computations on documents and dataframes.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize Spark**</font>
<hr style=" border:none; height:2px;">
</p>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar zxvf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:


import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/Users/viceroy/Downloads/content/spark-3.0.3-bin-hadoop2.7"
# os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-11.jdk/Contents/Home"

#os.environ["SPARK_HOME"] = "/Users/viceroy/Downloads/content/spark-3.0.1-bin-hadoop3.2"
#os.environ["SPARK_HOME"] = "/Users/viceroy/Downloads/spark-3.0.3-bin-hadoop2.7" 
#os.environ["SPARK_HOME"] = "/usr/local/lib/python3.8/site-packages/pyspark/"

import pandas as pd

import findspark
findspark.init("/content/spark-3.0.3-bin-hadoop2.7")

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("hw4") \
    .getOrCreate()


In [ ]:
!apt install unzip
!unzip lyrics_files_idioms.zip -d lyrics_files_idioms

# Analysing documents


<p align="justify">
<font size="3">
We have already seen that MapReduce procedures are good in analyzing text-files.
    
The provided data comes from a scraping operation on the website https://www.vagalume.com.br/ and is available on kaggle:
    
https://www.kaggle.com/neisse
    

    
The assignment is divided in 2 parts:
    
* Part 1 is focused on MapReduce 
    
* Part 2  is focuses on dataframes
    </font>
    </p>
    
<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that  dataset is noisy and shows all the typical issues related with data coming from this procedure (duplicated entries, etc).</font>
<hr style=" border:none; height:2px;">
</p>

# Part 1 -  MapReduce
<p align="justify">
<font size="3">
In the provided folder you can find a set of documents/files containing  descriptions of songs (lyrics and additional informations). Specifically in each file:

- the first line is the idiom/language
- the second line is the relative url of the song of the original website
- the third line is the title of a song 
- from fourth line on the text you find the lyrics of the song.
    </font>
    </p>

## Exercise 1 - (2 points) - Song's lyrics 

<p align="justify">
<font size="3">
Provide a Spark MapReduce procedure that reads the documents and checks how many song's lyrics appear at least two times.

In the data-interpretation of this exercise you can consider that two files represent the same lyric if the url (3rd line of each file) is the same.

 </font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that  you can reuse any code that was made available for the previous labs/assignments or that you already developed in these contexts.</font>
<hr style=" border:none; height:2px;">
</p>

In [16]:
### Write here your code
import re

#split by content in item[1]
def find_url(item): 
    description = item[1]
    lst = re.compile(r"\n").split(description)
    url = lst[1] #the second line is the relative url of the song of the original website
    return url, 1


In [20]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("hw4")
sc = SparkContext.getOrCreate(conf = conf)


In [ ]:
#note: took my computer 33 min to run
input_file = sc.wholeTextFiles("lyrics_files_idioms/*")
urls = input_file.map(find_url)
url_counts = urls.reduceByKey(lambda x, y: x + y)
duplicate_songs = url_counts.filter(lambda x: x[1] > 1)

In [50]:
print(f"Total Duplicates count: {duplicate_songs.count()}")

Total Duplicate count: 38096


## Exercise 2

### 2.1 - (1 point) - Distinct songs
Provide a Spark MapReduce procedure that provides how many distinct song's lyrics are present.

Also in this case consider the uri as key: two files represent the same lyric if the url is equal.

### 2.2 - (1 point) - Chaining
According to your implementation of Exercise 1, can you chain MapReduce additional MapReduce steps for solving Exercise 2.2? 

Provide the code for 2.1 and anwer for 2.2

In [51]:
### Write here your code followed by the answer to question 2.2

print(f"Total Distinct count: {url_counts.count()}")

Total Distinct count: 167499


# Exercise 3

### 3.1 - (3 points) - Most common word for language

Now that you discovered the duplicated documents consider just one occurence of each song's lyric and define a MapReduce procedure that finds the most common word for each language (of course you must remove stop words).




In [78]:
df = input_file.toDF()
df.show(5)

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|file:/content/lyr...|ENGLISH
/pac-div/...|
|file:/content/lyr...|ENGLISH
/johnny-m...|
|file:/content/lyr...|ENGLISH
/ian-dury...|
|file:/content/lyr...|ENGLISH
/flo-rida...|
|file:/content/lyr...|PORTUGUESE
/davi-...|
+--------------------+--------------------+
only showing top 5 rows



In [79]:
### Write here your code

from pyspark.ml.feature import StopWordsRemover 
stopwords = StopWordsRemover.loadDefaultStopWords("english")
#new_df = [remover.transform(i) for txt in df]
remover = StopWordsRemover(stopWords=stopwords, inputCol="_1", outputCol="_3")
#remover.transform(df)

IllegalArgumentException: ignored

### 3.2 - (3 points) - Most common end/start words

Finally discover, for each language, the most common ending and starting word (of course, also in this case) you must remove stop words).

In [ ]:
### Write here your code














<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**DataFrames**</font>
<hr style=" border:none; height:2px;">
</p>

# Part 2 - Dataframes

In this part you can use Pandas Dataframes or  Spark Dataframes.  I suggest to use a Spark Dataframe
end exploit the Pandas functionalities as we have seen in the 2nd assignment. Download the two available datasets at the link:

https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres

You can find two .cvs files: 

* artists-data.csv

* lyrics-data.csv


# Import artist data.
<p align="justify">
<font size="3">
The artist data in the .csv file can be stored in a dataframe. 
    
Each row of the .csv file describes an artist and the columns represent the following data:
    
* Artist - The artist's name
* Popularity - Popularity score at the date of scrapping
* ALink - The link to the artist's page
* AGenre - Primary musical genre of the artist
* AGenres - A list (pay attention to the format) of genres the artist fits in
    
</font>
</p>


# Import song's lyrics data.


<p align="justify">
<font size="3">
    
Each row of the .csv file describes a lyric and the columns represent the following data:
    
* ALink - The link to the webpage of the artist
* SLink - The link to the webpage of the song
* Idiom - The idiom of the lyric
* Lyric - The lyrics
* SName - The name of the song

    

</font>
</p>


#  Exercise 4 - (3 points) - Artist's genre

<p align="justify">
<font size="3">
Provide a program that finds the artists for which the genre is not specified.

</font>
</p>






In [80]:
### Write here your code

from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession \
    .builder \
    .appName("HW4") \
    .getOrCreate()

In [81]:
artists_df = spark.read.option("header", "true")\
    .option("inferSchema", "true")\
    .csv("artists-data.csv")

artists_df.show(10)


+-----------------+-----+----------+-------------------+-----+--------------------+
|           Artist|Songs|Popularity|               Link|Genre|              Genres|
+-----------------+-----+----------+-------------------+-----+--------------------+
|    10000 Maniacs|  110|       0.3|    /10000-maniacs/| Rock|Rock; Pop; Electr...|
|        12 Stones|   75|       0.3|        /12-stones/| Rock|Rock; Gospel/Reli...|
|              311|  196|       0.5|              /311/| Rock|Rock; Surf Music;...|
|    4 Non Blondes|   15|       7.5|    /4-non-blondes/| Rock|Rock; Pop/Rock; R...|
|A Cruz Está Vazia|   13|       0.0|/a-cruz-esta-vazia/| Rock|                Rock|
|  Aborto Elétrico|   36|       0.1|  /aborto-eletrico/| Rock|Rock; Punk Rock; ...|
|            Abril|   36|       0.1|            /abril/| Rock|Rock; Emocore; Ha...|
|            Abuse|   13|       0.0|            /abuse/| Rock|      Rock; Hardcore|
|            AC/DC|  192|      10.8|            /ac-dc/| Rock|Rock; Heavy Me

In [37]:
#we can see there is nothing == the data is very clean
artists_df.filter("Genre IS NULL").show(10)

+------+-----+----------+----+-----+------+
|Artist|Songs|Popularity|Link|Genre|Genres|
+------+-----+----------+----+-----+------+
+------+-----+----------+----+-----+------+



#  Exercise 5 - (3 points) - Duplicates

<p align="justify">
<font size="3">
Provide a program that removes the duplicates in the artists (also in this case the URL is the key).

</font>
</p>




In [33]:
### Write here your code
print((artists_df.count(), len(artists_df.columns))) #df.shape format

(3242, 6)


In [34]:
artists_new = artists_df.dropDuplicates(["Link"]) #url==Link is key
print((artists_new.count(), len(artists_new.columns)))

(2940, 6)


#  Exercise 6 - (4 points)

<p align="justify">
<font size="3">
Provide a program that using dataframe return the 100 most popular artists and the lyrics of their songs.
</font>
</p>

In [71]:
lyrics_df = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("lyrics-data.csv")

lyrics_df.show(10)

+---------------+--------------------+--------------------+--------------------+--------------------+
|          ALink|               SName|               SLink|               Lyric|               Idiom|
+---------------+--------------------+--------------------+--------------------+--------------------+
|/10000-maniacs/|      More Than This|/10000-maniacs/mo...|I could feel at t...|             ENGLISH|
|/10000-maniacs/|   Because The Night|/10000-maniacs/be...|Take me now, baby...|             ENGLISH|
|/10000-maniacs/|      These Are Days|/10000-maniacs/th...|These are. These ...|             ENGLISH|
|/10000-maniacs/|     A Campfire Song|/10000-maniacs/a-...|"A lie to say, ""...| ""O my. river wh...|
|/10000-maniacs/|Everyday Is Like ...|/10000-maniacs/ev...|Trudging slowly o...|             ENGLISH|
|/10000-maniacs/|          Don't Talk|/10000-maniacs/do...|Don't talk, I wil...|             ENGLISH|
|/10000-maniacs/|   Across The Fields|/10000-maniacs/ac...|Well they left th...|  

In [72]:
### Write here your code

sorted_artists = artists_new.sort(artists_new.Popularity.desc())\
    .limit(100)\
    .select("Artist", "Link")

sorted_artists.show(10)

+---------------+-----------------+
|         Artist|             Link|
+---------------+-----------------+
|  Ariana Grande|  /ariana-grande/|
|      Lady Gaga|      /lady-gaga/|
|       Maroon 5|       /maroon-5/|
|         Eminem|         /eminem/|
|          Queen|          /queen/|
|Imagine Dragons|/imagine-dragons/|
|          Adele|          /adele/|
|    The Beatles|    /the-beatles/|
|        Beyoncé|        /beyonce/|
|         Anitta|         /anitta/|
+---------------+-----------------+
only showing top 10 rows



In [73]:
combined = sorted_artists.join(lyrics_df, 
                               sorted_artists.Link == lyrics_df.ALink, 
                               "inner")\
    .select("Artist", "Sname", "Lyric")\
    
combined.show(10)

+------+--------------------+--------------------+
|Artist|               Sname|               Lyric|
+------+--------------------+--------------------+
| AC/DC|       Back In Black|Back in black, I ...|
| AC/DC|     Highway To Hell|Living easy, livi...|
| AC/DC|       Thunderstruck|Ahh-ahh-ahh-ahh-a...|
| AC/DC|              T.N.T.|Aye, aye, aye, ay...|
| AC/DC|You Shook Me All ...|She was a fast ma...|
| AC/DC| Rock 'n' Roll Train|One hot angel. On...|
| AC/DC|     Shoot To Thrill|All you women who...|
| AC/DC|         Hells Bells|I'm a rolling thu...|
| AC/DC|It's A Long Way T...|Ridin' down the h...|
| AC/DC|Dirty Deeds, Done...|If you're havin' ...|
+------+--------------------+--------------------+
only showing top 10 rows



# 2 - Bonus 


<p align="justify">
<font size="3">
Using the approach you prefer (just Dataframes, hybrid approach)  :
    
* the 10 most common words in the lyrics of each artist
* the 10 most common words for each genre. For this question we can use the primary genre of the artist.

</font>
</p>




In [ ]:
# Write here your code and the detailed description of the MapReduce algorithm.




















